##Set Up 

In [1]:
import torch
from torchvision.io import read_image
from torchvision import transforms
from PIL import Image                                 
import torchvision.transforms.functional as TF       
import numpy as np
import matplotlib.pyplot as plt
import os
from os import listdir

In [2]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/10_airliner.jpg  
  inflating: dataset/11_butterfly.jpg  
  inflating: dataset/12_butterfly.jpg  
  inflating: dataset/13_butterfly.jpg  
  inflating: dataset/14_butterfly.jpg  
  inflating: dataset/15_bird.jpg     
  inflating: dataset/16_bird.jpg     
  inflating: dataset/17_bird.jpg     
  inflating: dataset/18_bird.jpg     
  inflating: dataset/19_bird.jpg     
  inflating: dataset/1_airliner.jpg  
  inflating: dataset/20_bird.jpg     
  inflating: dataset/21_bird.jpg     
  inflating: dataset/22_bird.jpg     
  inflating: dataset/23_bird.jpg     
  inflating: dataset/24_bird.jpg     
  inflating: dataset/25_house.jpg    
  inflating: dataset/26_dog.jpg      
  inflating: dataset/27_dog.jpg      
  inflating: dataset/28_dog.jpg      
  inflating: dataset/29_dog.jpg      
  inflating: dataset/2_airliner.jpg  
  inflating: dataset/30_dog.jpg      
  inflating: dataset/31_dog.jpg      
  inflating: dataset/32_dog.jpg    

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [9]:
def get_acc(model):
  folder_dir = "/content/dataset"
  top_5_acc_num = 0 
  top_1_acc_num = 0 
  num_of_pics = 0
  errors = 0
  for image in os.listdir(folder_dir):
        num_of_pics = num_of_pics + 1
        if (image.endswith(".jpg")):
            img = Image.open(folder_dir+'/'+image)
            list_of_top_5_preds = model(img)
            print(image)
            print(list_of_top_5_preds)
            real_name = (image[image.find('_')+1:image.find('.')])
            if real_name in list_of_top_5_preds:
                top_5_acc_num = top_5_acc_num + 1
            if real_name==list_of_top_5_preds[0]:
                top_1_acc_num = top_1_acc_num + 1
  return top_5_acc_num/num_of_pics , top_1_acc_num/num_of_pics

## AlexNet

In [17]:
from torchvision.models import alexnet, AlexNet_Weights

In [18]:
weights_alexnet = AlexNet_Weights.IMAGENET1K_V1
model = alexnet(weights_alexnet)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [19]:
def alexnet(image): 
  image = preprocess(image)         
  img = image.unsqueeze(0) 
  if torch.cuda.is_available():
    img = img.to('cuda')
    model.to('cuda')
  prediction = model(img).squeeze(0).softmax(0)
  n = 5
  cat_names = []
  class_id = torch.topk(prediction.flatten(), n).indices
  for i in range(n):
    score = prediction[class_id[i]].item()
    category_name = weights.meta["categories"][class_id[i]]
    print(f"{category_name}: {100 * score:.1f}%")
    cat_names.append(category_name)
  return cat_names

In [20]:
%%time
t_5_acc, t_1_acc = get_acc(alexnet)
print(f'Top 5 accuracy:{np.round(t_5_acc,3)}, Top 1 accuracy:{np.round(t_1_acc,3)}')

tabby: 28.5%
hamper: 13.2%
Persian cat: 9.0%
cup: 7.2%
plastic bag: 6.8%
47_cat.jpg
['tabby', 'hamper', 'Persian cat', 'cup', 'plastic bag']
starfish: 99.7%
walking stick: 0.3%
wolf spider: 0.0%
mantis: 0.0%
king crab: 0.0%
43_starfish.jpg
['starfish', 'walking stick', 'wolf spider', 'mantis', 'king crab']
goldfinch: 26.0%
toucan: 10.1%
hornbill: 6.2%
bulbul: 5.8%
house finch: 5.6%
22_bird.jpg
['goldfinch', 'toucan', 'hornbill', 'bulbul', 'house finch']
brambling: 8.5%
black grouse: 7.2%
table lamp: 4.7%
squirrel monkey: 3.5%
wall clock: 3.2%
23_bird.jpg
['brambling', 'black grouse', 'table lamp', 'squirrel monkey', 'wall clock']
starfish: 100.0%
tiger shark: 0.0%
hammerhead: 0.0%
common newt: 0.0%
albatross: 0.0%
41_starfish.jpg
['starfish', 'tiger shark', 'hammerhead', 'common newt', 'albatross']
starfish: 99.9%
nail: 0.1%
knot: 0.0%
king crab: 0.0%
screw: 0.0%
42_starfish.jpg
['starfish', 'nail', 'knot', 'king crab', 'screw']
tiger cat: 54.4%
tabby: 41.7%
Egyptian cat: 3.5%
lynx: 0.

In [21]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Tesla T4
Memory Usage:
Allocated: 0.2 GB
Cached:    0.4 GB


In [22]:
print(round(torch.cuda.max_memory_allocated()/1024**3,1), 'GB')
if device == 'cuda':
    torch.cuda.reset_max_memory_allocated()

0.2 GB


### VGG19

In [ ]:
from torchvision.models import vgg19
from torchvision.models.vgg import VGG19_Weights

In [ ]:
weights_vgg19 = VGG19_Weights.IMAGENET1K_V1
model = vgg19(weights=weights_vgg19)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
def vgg19(image): 
  image = preprocess(image)          
  img = image.unsqueeze(0)   
  if torch.cuda.is_available():
    img = img.to('cuda')
    model.to('cuda')
  prediction = model(img).squeeze(0).softmax(0)
  n = 5
  cat_names = []
  class_id = torch.topk(prediction.flatten(), n).indices
  for i in range(n):
    score = prediction[class_id[i]].item()
    category_name = weights.meta["categories"][class_id[i]]
    print(f"{category_name}: {100 * score:.1f}%")
    cat_names.append(category_name)
  return cat_names

In [ ]:
%%time
t_5_acc, t_1_acc = get_acc(vgg19)
print(f'Top 5 accuracy:{np.round(t_5_acc,3)}, Top 1 accuracy:{np.round(t_1_acc,3)}')

space bar: 59.2%
computer keyboard: 21.7%
typewriter keyboard: 8.9%
hand-held computer: 7.2%
mouse: 1.2%
template24_keyboard.jpg
['space bar', 'computer keyboard', 'typewriter keyboard', 'hand-held computer', 'mouse']
space bar: 47.3%
computer keyboard: 19.3%
typewriter keyboard: 8.2%
crossword puzzle: 3.8%
remote control: 1.9%
template25_keyboard.jpg
['space bar', 'computer keyboard', 'typewriter keyboard', 'crossword puzzle', 'remote control']
crate: 50.7%
binder: 20.3%
mousetrap: 6.4%
maze: 3.9%
carton: 3.8%
template1_box.jpg
['crate', 'binder', 'mousetrap', 'maze', 'carton']
mixing bowl: 36.7%
croquet ball: 8.8%
sunscreen: 7.4%
pencil sharpener: 5.7%
face powder: 4.4%
template3_box.jpg
['mixing bowl', 'croquet ball', 'sunscreen', 'pencil sharpener', 'face powder']
flagpole: 15.1%
patio: 13.2%
lawn mower: 10.7%
library: 7.4%
barrow: 6.8%
template31_house.jpg
['flagpole', 'patio', 'lawn mower', 'library', 'barrow']
mobile home: 61.5%
boathouse: 15.4%
patio: 10.7%
window screen: 2.8%


In [15]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Tesla T4
Memory Usage:
Allocated: 0.1 GB
Cached:    0.2 GB


In [16]:
print(round(torch.cuda.max_memory_allocated()/1024**3,1), 'GB')
if device == 'cuda':
    torch.cuda.reset_max_memory_allocated()

0.2 GB


### ResNet50

In [10]:
from torchvision.models import resnet50, ResNet50_Weights

In [11]:
weights = ResNet50_Weights.IMAGENET1K_V1
model = resnet50(weights=weights)
model.eval()
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),
])

In [12]:
def resnet(img):
  image = preprocess(img).unsqueeze(0)  

  if torch.cuda.is_available():
    image = image.to('cuda')
    model.to('cuda')
        
  prediction = model(image).squeeze(0).softmax(0)
  n = 5
  class_id = torch.topk(prediction.flatten(), n).indices
  cat_names = []
  for i in range(n):
    score = prediction[class_id[i]].item()
    category_name = weights.meta["categories"][class_id[i]]
    print(f"{category_name}: {100 * score:.1f}%")
    cat_names.append(category_name)
  return cat_names

In [13]:
%%time
t_5_acc, t_1_acc = get_acc(resnet)
print(f'Top 5 accuracy:{np.round(t_5_acc,3)}, Top 1 accuracy:{np.round(t_1_acc,3)}')

lynx: 33.4%
Persian cat: 26.8%
tabby: 12.4%
tiger cat: 12.2%
Egyptian cat: 7.7%
47_cat.jpg
['lynx', 'Persian cat', 'tabby', 'tiger cat', 'Egyptian cat']
starfish: 100.0%
sea urchin: 0.0%
coral reef: 0.0%
stinkhorn: 0.0%
nail: 0.0%
43_starfish.jpg
['starfish', 'sea urchin', 'coral reef', 'stinkhorn', 'nail']
goldfinch: 31.7%
bee eater: 12.0%
brambling: 11.3%
kite: 10.3%
house finch: 8.7%
22_bird.jpg
['goldfinch', 'bee eater', 'brambling', 'kite', 'house finch']
black grouse: 48.4%
jacamar: 17.9%
water ouzel: 9.3%
prairie chicken: 7.2%
kite: 3.5%
23_bird.jpg
['black grouse', 'jacamar', 'water ouzel', 'prairie chicken', 'kite']
starfish: 100.0%
coral reef: 0.0%
sea urchin: 0.0%
brain coral: 0.0%
sea cucumber: 0.0%
41_starfish.jpg
['starfish', 'coral reef', 'sea urchin', 'brain coral', 'sea cucumber']
starfish: 100.0%
sea urchin: 0.0%
stinkhorn: 0.0%
sea cucumber: 0.0%
earthstar: 0.0%
42_starfish.jpg
['starfish', 'sea urchin', 'stinkhorn', 'sea cucumber', 'earthstar']
tiger cat: 49.6%
Egyp

In [14]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Tesla T4
Memory Usage:
Allocated: 0.1 GB
Cached:    0.2 GB


/usr/local/lib/python3.8/dist-packages/torch/cuda/memory.py:395: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [ ]:
print(round(torch.cuda.max_memory_allocated()/1024**3,1), 'GB')
if device == 'cuda':
    torch.cuda.reset_max_memory_allocated()

0.2 GB
